In [1]:
import random
import math
import torch
import torch.nn as nn
import torch.optim as optim

from ModelParallel import ModelParallel, get_device_free_memory

import time

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [3]:
devices = [device]

In [4]:
batch_size = 32

In [5]:
X = [random.random() * 10 for i in range(1000000)]

In [6]:
y = [math.exp(x) for x in X]

In [7]:
X = torch.tensor(X,dtype=torch.float).unsqueeze(-1)

In [8]:
y = torch.tensor(y,dtype=torch.float).unsqueeze(-1)

# SIGNLE-GPU

In [ ]:
class ModelP(ModelParallel):
  def __init__(self):
    super(ModelP, self).__init__()
    self.layer1 = nn.Linear(1, 102,bias=False)
    self.layer2 = nn.Linear(102, 102,bias=False)
    self.layer3 = nn.Linear(102, 102,bias=False)
    self.layer4 = nn.Linear(102, 102,bias=False)
    self.layer5 = nn.Linear(102, 102,bias=False)
    self.layer6 = nn.Linear(102, 102,bias=False)
    self.layerLast = nn.Linear(102, 1,bias=False)

  def forward(self, x):
    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)
    x = self.layer4(x)
    x = self.layer5(x)
    x = self.layer6(x)
    x = self.layerLast(x)
    return x

In [ ]:
model = ModelP()
model.to(device)

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
model.train()

t1 = time.time()

nb_batch = 0

for epoch in range(1):
  mae = 0
  nb_batch = 0
  for idx in range(0, len(X) - batch_size, batch_size):
    features = X[idx:idx+batch_size,:]
    #targets = torch.tensor(y[idx:idx+batch_size],dtype=torch.float).to(device)

    pred = model(features)

    targets = y[idx:idx+batch_size,:].to(pred.device)

    loss = criterion(pred, targets)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    mae += loss.item()
    nb_batch += 1

  print("epoch {} - mae {}".format(epoch, mae / nb_batch))

t2 = time.time()
(t2 - t1) / nb_batch

# MULTI-GPU

In [9]:
class ModelP(ModelParallel):
  def __init__(self):
    super(ModelP, self).__init__()
    self.layer1 = self.mp_m(nn.Linear(1, 102,bias=False), 0)
    self.layer2 = self.mp_m(nn.Linear(102, 102,bias=False), 1)
    self.layer3 = self.mp_m(nn.Linear(102, 102,bias=False), 0.5)
    self.layer4 = self.mp_m(nn.Linear(102, 102,bias=False), 0.5)
    self.layer5 = self.mp_m(nn.Linear(102, 102,bias=False), 1)
    self.layer6 = self.mp_m(nn.Linear(102, 102,bias=False), 1)
    self.layerLast = self.mp_m(nn.Linear(102, 1,bias=False), 0)

  def forward(self, x):
    x = self.mp_f(self.layer1, x)
    x = self.mp_f(self.layer2, x)
    x = self.mp_f(self.layer3, x)
    x = self.mp_f(self.layer4, x)
    x = self.mp_f(self.layer5, x)
    x = self.mp_f(self.layer6, x)
    x = self.mp_f(self.layerLast, x)
    return x

In [10]:
model = ModelP()
model.to_devices(devices)

In [11]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [12]:
model.train()

t1 = time.time()

nb_batch = 0

for epoch in range(1):
  mae = 0
  nb_batch = 0
  for idx in range(0, len(X) - batch_size, batch_size):
    features = X[idx:idx+batch_size,:]
    #targets = torch.tensor(y[idx:idx+batch_size],dtype=torch.float).to(device)

    pred = model(features)

    targets = y[idx:idx+batch_size,:].to(pred.device)

    loss = criterion(pred, targets)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    mae += loss.item()
    nb_batch += 1

  print("epoch {} - mae {}".format(epoch, mae / nb_batch))

t2 = time.time()
(t2 - t1) / nb_batch

KeyboardInterrupt: 

In [ ]:
for dev in devices:
  print(dev, get_device_free_memory(dev))